<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_%EA%B0%9C%EC%A0%952%ED%8C%90_ch8_7_p_538_549.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 07 문서 군집화 소개와 실습（Opinion Review 데이터 세트）

- **문서 군집화(Document Clustering)**:  
  비슷한 텍스트 구성을 가진 문서들을 하나의 군집으로 묶는 작업

- **문서 분류와의 차이점**:  
  - 텍스트 분류: 사전에 정의된 카테고리와 학습 데이터가 필요 (지도학습)
  - 문서 군집화: 사전 정보 없이 문서 자체의 특성만으로 군집을 형성 (비지도학습)


## Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

- 사용 데이터: UCI 머신러닝 리포지토리의 Opinion Review 데이터 세트

- 구성: 총 51개의 텍스트 파일, 각 문서는 약 100개의 문장 포함

- 출처: Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품)

- 목적: 문서 군집화를 통해 리뷰 문서를 분류

- 참고: 원래는 토픽 모델링 논문에 사용된 데이터 세트

In [13]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',700)

# ✅ 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/25-1 ESAA OB/OpinosisDataset1.0/topics'

# path로 지정한 디렉터리 밑의 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    # 절대경로로 주어진 파일명을 가공
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi..."
1,free_bestwestern_hotel_sfo,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...
2,features_windows7,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ..."
3,location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ..."
4,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...


In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def LemNormalize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    return [wordnet_lemmatizer.lemmatize(token) for token in text.split()]

[nltk_data] Downloading package wordnet to /root/nltk_data...


### TF-IDF 피처 벡터화

- **사용 도구**: `TfidfVectorizer`
- **어근 변환**:  
  - `TfidfVectorizer`는 어근 변환(Lemmatization)을 기본 지원하지 않음  
  - `tokenizer` 인자에 커스텀 함수(`LemNormalize`)를 사용해 어근 변환 수행
- **설정 값**:  
  - `ngram_range=(1, 2)` : 단어와 2-그램 포함  
  - `min_df`, `max_df` 설정으로 피처 수 제한
- **적용 대상**:  
  - `document_df`의 `opinion_text` 컬럼에 `fit_transform()` 적용  
  - 결과: 각 문서가 **TF-IDF 피처 벡터**로 변환된 행렬 생성


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1, 2), min_df=0.05, max_df=0.85)

# opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

### K-평균을 활용한 문서 군집화

- **입력 데이터**:  
  TF-IDF로 피처 벡터화된 문서 행렬

- **군집화 알고리즘**:  
  `KMeans` 알고리즘 사용

- **군집 수(K)**:  
  문서 유형을 고려해 **5개 중심(Centroid)**으로 설정  
  (예: 전자제품, 자동차, 호텔 등)

- **설정 값**:  
  - `max_iter=10000`: 최대 반복 횟수

- **출력 결과**:  
  - 각 문서의 **군집 라벨(Label)**  
  - 각 중심(Centroid)의 **좌표 값**


In [16]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

- `KMeans`로 생성된 각 문서의 **군집 레이블**을 `document_df` DataFrame에 `cluster_label` 컬럼으로 추가

- **목적**:  
  각 문서의 **파일명(주제)**과 **군집 레이블**을 함께 확인하여, **유사한 주제끼리 잘 군집되었는지** 평가

In [17]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi...",1
1,free_bestwestern_hotel_sfo,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,2
2,features_windows7,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",3
3,location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",2
4,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,1


- **정렬 방법**:  
  `document_df.sort_values(by='cluster_label')`  
  → `cluster_label` 기준으로 데이터 정렬

- **목적**:  
  각 군집(cluster_label)에 어떤 **파일명(문서 주제)**이 할당되었는지 확인하여 군집화 결과의 유사성 및 주제별 분류 상태 파악

In [18]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
20,comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",0
5,comfort_toyota_camry_2007,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",0
16,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,0
15,interior_honda_accord_2008,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,0
21,interior_toyota_camry_2007,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",0
8,mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",0
37,performance_honda_accord_2008,"Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long, distance cruiser .\n0 6, 4, 3 eco engine has poor performance and gas mileage of 22 highway .\n1 Overall performance is good but comfort level is poor .\n2 ...",0
35,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,0
27,seats_honda_accord_2008,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",0
39,transmission_toyota_camry_2007,"After slowing down, transmission has to be kicked to speed up .\n0 ...",0


- Cluster #0 : 토요타(Toyota)와 혼다(Honda) 등의 자동차에 대한 리뷰로 군집화

In [19]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi...",1
4,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,1
28,satellite_garmin_nuvi_255W_gps,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",1
46,sound_ipod_nano_8gb,headphone jack i got a clear case for it and it i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1 ...,1
26,updates_garmin_nuvi_255W_gps,"Another thing to consider was that I paid $50 less for the 750 and it came with the FM transmitter cable and a USB cord to connect it to your computer for updates and downloads .\n0 update and reroute much _more_ quickly than my other GPS .\n1 UPDATE ON THIS , It finally turned out that to see the elevation contours at lowe...",1
31,voice_garmin_nuvi_255W_gps,The voice prompts and maps are wonderful especially when driving after dark .\n0 I also thought the the voice prompts of the 750 where more pleasant sounding than the 255w's .\n1 ...,1


- Cluster #1 : 차량용내비게이션으로 군집화

In [20]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
24,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",2
6,food_holiday_inn_london,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,2
18,food_swissotel_chicago,The food for our event was delicious .\n0 ...,2
1,free_bestwestern_hotel_sfo,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,2
3,location_bestwestern_hotel_sfo,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",2
22,location_holiday_inn_london,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",2
19,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,2
29,room_holiday_inn_london,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",2
30,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , H...",2
47,rooms_swissotel_chicago,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",2


- Cluster #2 : 호텔에 대한 리뷰로 군집화

In [21]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
12,battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ...",3
10,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,3
14,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now...",3
11,display_garmin_nuvi_255W_gps,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",3
2,features_windows7,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",3
9,keyboard_netbook_1005ha,", I think the new keyboard rivals the great hp mini keyboards .\n0 Since the battery life difference is minimum, the only reason to upgrade would be to get the better keyboard .\n1 The keyboard is now as good as t...",3
41,performance_netbook_1005ha,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",3
33,screen_garmin_nuvi_255W_gps,It is easy to read and when touching the screen it works great !\n0 and zoom out buttons on the 255w to the same side of the screen which makes it a bit easier .\n1 ...,3
36,screen_ipod_nano_8gb,"As always, the video screen is sharp and bright .\n0 2, inch screen and a glossy, polished aluminum finish that one CNET editor described as looking like a Christmas tree ornament .\n1 ...",3
49,screen_netbook_1005ha,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,3


- Cluster #3 : 킨들, 아이팟, 넷북 등의 포터블 전자기기 및 주요 구성요소(배터리, 키보드 등)에 대한 리뷰로 군집화

In [22]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
13,buttons_amazon_kindle,"I thought it would be fitting to christen my Kindle with the Stephen King novella UR, so went to the Amazon site on my computer and clicked on the button to buy it .\n0 As soon as I'd clicked the button to confirm my order it appeared on my Kindle almost immediately !\n1 ...",4
7,eyesight-issues_amazon_kindle,"It feels as easy to read as the K1 but doesn't seem any crisper to my eyes .\n0 the white is really GREY, and to avoid considerable eye, strain I had to refresh pages every other page .\n1 The dream has always been a portable electronic device that could hold a ton of reading material, automate subscriptions and fa...",4
23,fonts_amazon_kindle,"Being able to change the font sizes is awesome !\n0 For whatever reason, Amazon decided to make the Font on the Home Screen ...",4
17,navigation_amazon_kindle,"In fact, the entire navigation structure has been completely revised , I'm still getting used to it but it's a huge step forward .\n0 ...",4
25,price_amazon_kindle,"If a case was included, as with the Kindle 1, that would have been reflected in a higher price .\n0 lower overall price, with nice leather cover .\n1 ...",4
45,price_holiday_inn_london,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",4


- Cluster #3 : 대부분 호텔에 대한 리뷰로 군집화

- 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화

In [29]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
5,comfort_toyota_camry_2007,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",0
8,mileage_honda_accord_2008,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",0
15,interior_honda_accord_2008,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,0
16,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,0
27,seats_honda_accord_2008,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",0
21,interior_toyota_camry_2007,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",0
20,comfort_honda_accord_2008,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",0
35,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,0
41,performance_netbook_1005ha,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",0
39,transmission_toyota_camry_2007,"After slowing down, transmission has to be kicked to speed up .\n0 ...",0


- Cluster #0 : 자동차 리뷰로만 군집 구성
- Cluster #1 : 포터블 전자기기 리뷰로만 군집화
- Cluster #2 : 호텔 리뷰로만 군집 구성



## 군집별 핵심 단어 추출하기

- **활용 속성**:  
  `KMeans` 객체의 `cluster_centers_` 속성 사용

- **설명**:  
  - `cluster_centers_`는 **군집 중심 좌표** 정보를 담고 있음  
  - **형태**: 2차원 배열 (행: 군집, 열: TF-IDF 피처)  
  - 각 값은 해당 피처(단어)가 군집 중심과 얼마나 가까운지를 나타냄  
    → 값이 클수록 **군집의 핵심 단어**

- **예시**:  
  `km_cluster.cluster_centers_[0, 1]`  
  → **0번 군집**에서 **두 번째 단어**의 중심 좌표 값

- **목적**:  
  각 군집의 중심에서 가장 큰 값을 갖는 단어를 추출해  
  **해당 군집의 주제를 설명하는 핵심 키워드**로 사용

In [30]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 5648)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.00178265 0.00131867 0.         ... 0.         0.         0.        ]
 [0.         0.00043948 0.00099222 ... 0.00197    0.00156757 0.00156757]]


- cluster_centers shape: (3, 5648) -> 군집이 3개, word 피처가 5648개로 구성

### 군집별 핵심 단어 추출 함수: `get_cluster_details()`

- **목적**:  
  각 군집의 중심에서 **가장 영향력 있는 핵심 단어와 그 값**을 추출

- **사용 데이터**:  
  `KMeans.cluster_centers_` → 각 군집 중심의 TF-IDF 피처 값  
  `TfidfVectorizer.get_feature_names_out()` → 피처 이름(단어)

- **핵심 로직**:
  1. `cluster_centers_`에서 **값이 큰 순서대로 인덱스**를 정렬  
     - `argsort()[:, ::-1]` 사용
  2. 정렬된 인덱스를 통해 **핵심 단어의 피처 이름**과 **중심값** 추출
  3. 이를 반복하여 각 군집별로 핵심 단어 리스트를 생성
  4. 결과를 `cluster_details`라는 딕셔너리에 저장 및 반환

In [38]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num,
                        top_n_features=10):
  cluster_details = {}

  #cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(cluster_num):
    # 개별 군집별 정보를 담을 데이터 초기화.
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함.
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames = filenames.values.tolist()
    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

### 군집 핵심 단어 정보 출력 함수: `print_cluster_details()`

- **목적**:  
  `get_cluster_details()`로 반환된 각 군집의 핵심 단어 정보를 보기 좋게 출력하여 해석을 용이하게 함

- **입력값**:  
  - `cluster_details`: 각 군집에 대한 핵심 단어 및 정보가 담긴 리스트/딕셔너리
  - `document_df`: 각 문서의 파일명과 군집 레이블이 포함된 DataFrame

- **출력 내용**:
  - 군집 번호 (`Cluster n`)
  - 핵심 단어 및 중심 상대값 (상위 n개)
  - 해당 군집에 포함된 문서들의 파일명 목록


In [40]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('===============================================')

문서 군집화 결과 확인 절차


1. TF-IDF 벡터화 객체에서 피처명 리스트 추출


2. 군집별 핵심 단어 정보 추출


3. 군집별 핵심 단어 및 문서 출력



In [43]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                      feature_names=feature_names, cluster_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'performance', 'gas', 'gas mileage', 'comfortable', 'transmission', 'car', 'quality']
Reviews 파일명: ['comfort_toyota_camry_2007', 'mileage_honda_accord_2008', 'interior_honda_accord_2008', 'gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'seats_honda_accord_2008']
####### Cluster 1
Top features: ['screen', 'battery', 'keyboard', 'kindle', 'video', 'voice', 'direction', 'size', 'feature', 'map']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'features_windows7', 'directions_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'keyboard_netbook_1005ha', 'battery-life_ipod_nano_8gb', 'display_garmin_nuvi_255W_gps']
####### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'price', 'parking', 'clean']
Reviews 파일명: ['free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'parking_

- Cluster #0(자동차 리뷰 군집) : ‘interior’, ‘seat’, ‘mileage’, ‘comfortable’ 등과 같은 실내 인테리어, 좌석, 연료 효율 등이 핵심 단어로 군집화. 토요타, 혼다와 같은 일본 자동차의 경우 실내 인테리어와 연료 효율, 편안함이 주요 관심사

- Cluster #1 : screen’, ‘battery’, battery life’ 등과 같은 화면과 배터리 수명 등이 핵심 단어로 군집화. 아무래도 모바일형이고 엔터테인먼트용 전자제품
의 경우 화면 크기와 배터리 수명이 주요 관심사
- Cluster #2(호텔 리뷰 군집): 'room', ‘hotel’, 'service', 'staff' 등 같은
방과 서비스 등이 핵심 단어로 군집화. 호텔 리뷰의 경우 방의 크기나 청소 상태, 직원들의
서비스, 위치 등이 주요 관심사